In [2]:
import pyspark
import findspark
findspark.init()

In [3]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
spark=SparkSession.builder.appName('group by and aggregations').getOrCreate()
spark

In [4]:
import pandas as pd
data=[("Sona","Sales","NY",90000,34,10000),
      ("Mona","Sales","NY",86000,56,20000),
      ("Bannu","Sales","CA",81000,30,23000),
      ("Chinnu","Finance","CA",90000,24,23000),
    ("Sunny","Finance","CA",99000,40,24000),
    ("Raju","Finance","NY",83000,36,19000),
    ("ravi","Finance","NY",79000,53,15000),
    ("Dhana","Marketing","CA",80000,25,18000),
    ("Ram","Marketing","NY",91000,50,21000)]

columns=["Name","Department","State","Salary","Age","Bonus"]

df=pd.DataFrame(data,columns=columns)
df1=spark.createDataFrame(df)

In [11]:
print(type(df))
df

<class 'pandas.core.frame.DataFrame'>


,Name,Department,State,Salary,Age,Bonus
0,Sona,Sales,NY,90000,34,10000
1,Mona,Sales,NY,86000,56,20000
2,Bannu,Sales,CA,81000,30,23000
3,Chinnu,Finance,CA,90000,24,23000
4,Sunny,Finance,CA,99000,40,24000
5,Raju,Finance,NY,83000,36,19000
6,ravi,Finance,NY,79000,53,15000
7,Dhana,Marketing,CA,80000,25,18000
8,Ram,Marketing,NY,91000,50,21000


In [4]:
print(type(df1))

<class 'pyspark.sql.dataframe.DataFrame'>


In [5]:
df1.show()

+------+----------+-----+------+---+-----+
|  Name|Department|State|Salary|Age|Bonus|
+------+----------+-----+------+---+-----+
|  Sona|     Sales|   NY| 90000| 34|10000|
|  Mona|     Sales|   NY| 86000| 56|20000|
| Bannu|     Sales|   CA| 81000| 30|23000|
|Chinnu|   Finance|   CA| 90000| 24|23000|
| Sunny|   Finance|   CA| 99000| 40|24000|
|  Raju|   Finance|   NY| 83000| 36|19000|
|  ravi|   Finance|   NY| 79000| 53|15000|
| Dhana| Marketing|   CA| 80000| 25|18000|
|   Ram| Marketing|   NY| 91000| 50|21000|
+------+----------+-----+------+---+-----+



In [14]:
df1.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Department: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Salary: long (nullable = true)
 |-- Age: long (nullable = true)
 |-- Bonus: long (nullable = true)



In [23]:
# group by and aggregations
# 1) sum of salary department wise
df1.groupBy("Department").sum("Salary").show()

+----------+-----------+
|Department|sum(Salary)|
+----------+-----------+
|     Sales|     257000|
|   Finance|     351000|
| Marketing|     171000|
+----------+-----------+



In [21]:
# 2) number of employees dept wise
df1.groupBy("Department").count().show()

+----------+-----+
|Department|count|
+----------+-----+
|     Sales|    3|
|   Finance|    4|
| Marketing|    2|
+----------+-----+



In [24]:
# 3) Minimum salary for each department
df1.groupBy("Department").min("Salary").show()

+----------+-----------+
|Department|min(Salary)|
+----------+-----------+
|     Sales|      81000|
|   Finance|      79000|
| Marketing|      80000|
+----------+-----------+



In [25]:
# 4) Maximum salary of each dept
df1.groupBy("Department").max("Salary").show()

+----------+-----------+
|Department|max(Salary)|
+----------+-----------+
|     Sales|      90000|
|   Finance|      99000|
| Marketing|      91000|
+----------+-----------+



In [26]:
# 5) AVerage salary of each dept
df1.groupBy("Department").avg("Salary").show()

+----------+-----------------+
|Department|      avg(Salary)|
+----------+-----------------+
|     Sales|85666.66666666667|
|   Finance|          87750.0|
| Marketing|          85500.0|
+----------+-----------------+



In [6]:
# 6) Mean of salary dept wise
df1.groupBy("Department").mean("Salary").show()

+----------+-----------------+
|Department|      avg(Salary)|
+----------+-----------------+
|     Sales|85666.66666666667|
|   Finance|          87750.0|
| Marketing|          85500.0|
+----------+-----------------+



In [29]:
# 7) Sum of bonus given by each dept
df1.groupBy("Department").sum("Bonus").show()

+----------+----------+
|Department|sum(Bonus)|
+----------+----------+
|     Sales|     53000|
|   Finance|     81000|
| Marketing|     39000|
+----------+----------+



In [31]:
# 8) Summary statistics
df1.describe().show()

+-------+-----+----------+-----+-----------------+------------------+------------------+
|summary| Name|Department|State|           Salary|               Age|             Bonus|
+-------+-----+----------+-----+-----------------+------------------+------------------+
|  count|    9|         9|    9|                9|                 9|                 9|
|   mean| NULL|      NULL| NULL|86555.55555555556|38.666666666666664|19222.222222222223|
| stddev| NULL|      NULL| NULL|6540.472290116195|11.947803145348521| 4465.920335658087|
|    min|Bannu|   Finance|   CA|            79000|                24|             10000|
|    max| ravi|     Sales|   NY|            99000|                56|             24000|
+-------+-----+----------+-----+-----------------+------------------+------------------+



In [34]:
# group by multiple columns
# group by department,state and find sum of salary and bonus
df1.groupBy("Department","State").sum("Salary","Bonus").show()



+----------+-----+-----------+----------+
|Department|State|sum(Salary)|sum(Bonus)|
+----------+-----+-----------+----------+
|     Sales|   NY|     176000|     30000|
|     Sales|   CA|      81000|     23000|
|   Finance|   CA|     189000|     47000|
|   Finance|   NY|     162000|     34000|
| Marketing|   NY|      91000|     21000|
| Marketing|   CA|      80000|     18000|
+----------+-----+-----------+----------+



In [51]:
# Running more aggregates at a time
# Example-1
from pyspark.sql.functions import sum,avg,min,max,mean,count
df1.groupBy("Department").agg(sum("Salary").alias("sum_salary"),
            avg("Salary").alias("avg_salary"),
                min("Salary").alias("Min_salary"),
            max("Salary").alias("Max_Salary"),
            mean("Salary").alias("Mean Salary"),
            count("*").alias("Total count")).show()

+----------+----------+-----------------+----------+----------+-----------------+-----------+
|Department|sum_salary|       avg_salary|Min_salary|Max_Salary|      Mean Salary|Total count|
+----------+----------+-----------------+----------+----------+-----------------+-----------+
|     Sales|    257000|85666.66666666667|     81000|     90000|85666.66666666667|          3|
|   Finance|    351000|          87750.0|     79000|     99000|          87750.0|          4|
| Marketing|    171000|          85500.0|     80000|     91000|          85500.0|          2|
+----------+----------+-----------------+----------+----------+-----------------+-----------+



In [ ]:
# Example-2
# Aggregate functions using agg for bonus for each state
from pyspark.sql.functions import min,max,sum,avg,count,mean

In [52]:
df1.groupby("State").agg(sum("Bonus").alias("Bonus_sum"),
                        max("Bonus").alias("Bonus_max"),
                        min("Bonus").alias("Bonus_min"),
                        avg("Bonus").alias("Bonus_avg"),
                        mean("Bonus").alias("Bonus_mean"),
                        count("*").alias("Total count")).show()

+-----+---------+---------+---------+---------+----------+-----------+
|State|Bonus_sum|Bonus_max|Bonus_min|Bonus_avg|Bonus_mean|Total count|
+-----+---------+---------+---------+---------+----------+-----------+
|   NY|    85000|    21000|    10000|  17000.0|   17000.0|          5|
|   CA|    88000|    24000|    18000|  22000.0|   22000.0|          4|
+-----+---------+---------+---------+---------+----------+-----------+



In [49]:
# filtering the aggregated data
from pyspark.sql.functions import col
df1.groupby("State").agg(sum("Bonus").alias("Bonus_sum"),
                        max("Bonus").alias("Bonus_max"),
                        min("Bonus").alias("Bonus_min")).where(col("Bonus_min")>12000).show()

+-----+---------+---------+---------+
|State|Bonus_sum|Bonus_max|Bonus_min|
+-----+---------+---------+---------+
|   CA|    88000|    24000|    18000|
+-----+---------+---------+---------+



In [56]:
# Aggregations without group by
# 1) approx_count_distinct

# approx_count_distinct() function returns the count of distinct items in a group.
from pyspark.sql.functions import *
df1.select(approx_count_distinct("State")).collect()

[Row(approx_count_distinct(State)=2)]

In [61]:
# 2) collect_list Aggregate Function
#collect_list() function returns all values from an input column with duplicates.

df1.select(collect_list("Salary")).show(truncate=False)

+---------------------------------------------------------------+
|collect_list(Salary)                                           |
+---------------------------------------------------------------+
|[90000, 86000, 81000, 90000, 99000, 83000, 79000, 80000, 91000]|
+---------------------------------------------------------------+



In [62]:
#3) collect_set Aggregate Function
#collect_set() function returns all values from an input column with duplicate values eliminated.

df1.select(collect_set("Salary")).show(truncate=False)

+--------------------------------------------------------+
|collect_set(Salary)                                     |
+--------------------------------------------------------+
|[79000, 83000, 91000, 99000, 90000, 80000, 86000, 81000]|
+--------------------------------------------------------+



In [64]:
# 4) avg
# 5) count
df1.select(avg("Salary").alias("Avg_Salary")).show()
df1.select(count("*").alias("Total Count")).show()

+-----------------+
|       Avg_Salary|
+-----------------+
|86555.55555555556|
+-----------------+

+-----------+
|Total Count|
+-----------+
|          9|
+-----------+



In [66]:
# 6) first()------gives first element
# 7) last()-------gives last element
# 8) max()
# 9) min()
# 10) mean()
# 11) sum()
# 12) sumDistinct()
# 13) stddev()-----------Standard Deviation
# 14) variance()


df1.select(first("Salary").alias("First Salary")).show()
df1.select(last("Salary").alias("Last Salary")).show()
df1.select(max("Salary").alias("Max_Salary")).show()
df1.select(min("Salary").alias("Min_Salary")).show()
df1.select(mean("Salary").alias("Mean_Salary")).show()
df1.select(sum("Salary").alias("sum_Salary")).show()
df1.select(sum_distinct("Salary").alias("Sum of Salary of Distinct values")).show()
df1.select(stddev("Salary").alias("Standard Deviation_Salary")).show()
df1.select(variance("Salary").alias("Variance_Salary")).show()





+------------+
|First Salary|
+------------+
|       90000|
+------------+

+-----------+
|Last Salary|
+-----------+
|      91000|
+-----------+

+----------+
|Max_Salary|
+----------+
|     99000|
+----------+

+----------+
|Min_Salary|
+----------+
|     79000|
+----------+

+-----------------+
|      Mean_Salary|
+-----------------+
|86555.55555555556|
+-----------------+

+----------+
|sum_Salary|
+----------+
|    779000|
+----------+

+--------------------------------+
|Sum of Salary of Distinct values|
+--------------------------------+
|                          689000|
+--------------------------------+

+-------------------------+
|Standard Deviation_Salary|
+-------------------------+
|        6540.472290116195|
+-------------------------+

+-------------------+
|    Variance_Salary|
+-------------------+
|4.277777777777778E7|
+-------------------+



In [73]:
######## Dropping
#1) Drop single Column From DataFrame

# i)
df1.drop("Age").printSchema()
df1.drop("Age").show()

root
 |-- Name: string (nullable = true)
 |-- Department: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Salary: long (nullable = true)
 |-- Bonus: long (nullable = true)

+------+----------+-----+------+-----+
|  Name|Department|State|Salary|Bonus|
+------+----------+-----+------+-----+
|  Sona|     Sales|   NY| 90000|10000|
|  Mona|     Sales|   NY| 86000|20000|
| Bannu|     Sales|   CA| 81000|23000|
|Chinnu|   Finance|   CA| 90000|23000|
| Sunny|   Finance|   CA| 99000|24000|
|  Raju|   Finance|   NY| 83000|19000|
|  ravi|   Finance|   NY| 79000|15000|
| Dhana| Marketing|   CA| 80000|18000|
|   Ram| Marketing|   NY| 91000|21000|
+------+----------+-----+------+-----+



In [75]:
df1.show()

+------+----------+-----+------+---+-----+
|  Name|Department|State|Salary|Age|Bonus|
+------+----------+-----+------+---+-----+
|  Sona|     Sales|   NY| 90000| 34|10000|
|  Mona|     Sales|   NY| 86000| 56|20000|
| Bannu|     Sales|   CA| 81000| 30|23000|
|Chinnu|   Finance|   CA| 90000| 24|23000|
| Sunny|   Finance|   CA| 99000| 40|24000|
|  Raju|   Finance|   NY| 83000| 36|19000|
|  ravi|   Finance|   NY| 79000| 53|15000|
| Dhana| Marketing|   CA| 80000| 25|18000|
|   Ram| Marketing|   NY| 91000| 50|21000|
+------+----------+-----+------+---+-----+



In [78]:
# ii)

df1.drop(col("Salary")).printSchema()
df1.drop(col("Salary")).show()

root
 |-- Name: string (nullable = true)
 |-- Department: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Age: long (nullable = true)
 |-- Bonus: long (nullable = true)

+------+----------+-----+---+-----+
|  Name|Department|State|Age|Bonus|
+------+----------+-----+---+-----+
|  Sona|     Sales|   NY| 34|10000|
|  Mona|     Sales|   NY| 56|20000|
| Bannu|     Sales|   CA| 30|23000|
|Chinnu|   Finance|   CA| 24|23000|
| Sunny|   Finance|   CA| 40|24000|
|  Raju|   Finance|   NY| 36|19000|
|  ravi|   Finance|   NY| 53|15000|
| Dhana| Marketing|   CA| 25|18000|
|   Ram| Marketing|   NY| 50|21000|
+------+----------+-----+---+-----+



In [79]:
# iii)

df1.drop(df1.Name).printSchema()
df1.drop(df1.Name).show()

root
 |-- Department: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Salary: long (nullable = true)
 |-- Age: long (nullable = true)
 |-- Bonus: long (nullable = true)

+----------+-----+------+---+-----+
|Department|State|Salary|Age|Bonus|
+----------+-----+------+---+-----+
|     Sales|   NY| 90000| 34|10000|
|     Sales|   NY| 86000| 56|20000|
|     Sales|   CA| 81000| 30|23000|
|   Finance|   CA| 90000| 24|23000|
|   Finance|   CA| 99000| 40|24000|
|   Finance|   NY| 83000| 36|19000|
|   Finance|   NY| 79000| 53|15000|
| Marketing|   CA| 80000| 25|18000|
| Marketing|   NY| 91000| 50|21000|
+----------+-----+------+---+-----+



In [80]:
# 2) Dropping multiple columns
# i)
df1.drop("Name","Department").printSchema()
df1.drop("Name","Department").show()

root
 |-- State: string (nullable = true)
 |-- Salary: long (nullable = true)
 |-- Age: long (nullable = true)
 |-- Bonus: long (nullable = true)

+-----+------+---+-----+
|State|Salary|Age|Bonus|
+-----+------+---+-----+
|   NY| 90000| 34|10000|
|   NY| 86000| 56|20000|
|   CA| 81000| 30|23000|
|   CA| 90000| 24|23000|
|   CA| 99000| 40|24000|
|   NY| 83000| 36|19000|
|   NY| 79000| 53|15000|
|   CA| 80000| 25|18000|
|   NY| 91000| 50|21000|
+-----+------+---+-----+



In [81]:
# ii)
df1.drop(df1.Name,df1.Department).printSchema()
df1.drop(df1.Name,df1.Department).show()

root
 |-- State: string (nullable = true)
 |-- Salary: long (nullable = true)
 |-- Age: long (nullable = true)
 |-- Bonus: long (nullable = true)

+-----+------+---+-----+
|State|Salary|Age|Bonus|
+-----+------+---+-----+
|   NY| 90000| 34|10000|
|   NY| 86000| 56|20000|
|   CA| 81000| 30|23000|
|   CA| 90000| 24|23000|
|   CA| 99000| 40|24000|
|   NY| 83000| 36|19000|
|   NY| 79000| 53|15000|
|   CA| 80000| 25|18000|
|   NY| 91000| 50|21000|
+-----+------+---+-----+



In [83]:
# iii)
cols=["Name","Department"]
df1.drop(*cols).printSchema()
df1.drop(*cols).show()

root
 |-- State: string (nullable = true)
 |-- Salary: long (nullable = true)
 |-- Age: long (nullable = true)
 |-- Bonus: long (nullable = true)

+-----+------+---+-----+
|State|Salary|Age|Bonus|
+-----+------+---+-----+
|   NY| 90000| 34|10000|
|   NY| 86000| 56|20000|
|   CA| 81000| 30|23000|
|   CA| 90000| 24|23000|
|   CA| 99000| 40|24000|
|   NY| 83000| 36|19000|
|   NY| 79000| 53|15000|
|   CA| 80000| 25|18000|
|   NY| 91000| 50|21000|
+-----+------+---+-----+



In [84]:
############# Sorting
# sort() or orderBy() function of PySpark DataFrame to sort DataFrame by ascending or descending order 
# based on single or multiple columns

# 1) sort()

# i) sorting one column in ascending order
df1.sort("Salary",ascending=True).show()

+------+----------+-----+------+---+-----+
|  Name|Department|State|Salary|Age|Bonus|
+------+----------+-----+------+---+-----+
|  ravi|   Finance|   NY| 79000| 53|15000|
| Dhana| Marketing|   CA| 80000| 25|18000|
| Bannu|     Sales|   CA| 81000| 30|23000|
|  Raju|   Finance|   NY| 83000| 36|19000|
|  Mona|     Sales|   NY| 86000| 56|20000|
|  Sona|     Sales|   NY| 90000| 34|10000|
|Chinnu|   Finance|   CA| 90000| 24|23000|
|   Ram| Marketing|   NY| 91000| 50|21000|
| Sunny|   Finance|   CA| 99000| 40|24000|
+------+----------+-----+------+---+-----+



In [85]:
# ii) sorting one column in descending order

df1.sort("Salary",ascending=False).show()

+------+----------+-----+------+---+-----+
|  Name|Department|State|Salary|Age|Bonus|
+------+----------+-----+------+---+-----+
| Sunny|   Finance|   CA| 99000| 40|24000|
|   Ram| Marketing|   NY| 91000| 50|21000|
|  Sona|     Sales|   NY| 90000| 34|10000|
|Chinnu|   Finance|   CA| 90000| 24|23000|
|  Mona|     Sales|   NY| 86000| 56|20000|
|  Raju|   Finance|   NY| 83000| 36|19000|
| Bannu|     Sales|   CA| 81000| 30|23000|
| Dhana| Marketing|   CA| 80000| 25|18000|
|  ravi|   Finance|   NY| 79000| 53|15000|
+------+----------+-----+------+---+-----+



In [86]:
# iii) sorting two columns one in ascending and other in descending

df1.sort("Salary","Age",ascending=[True,False]).show()

+------+----------+-----+------+---+-----+
|  Name|Department|State|Salary|Age|Bonus|
+------+----------+-----+------+---+-----+
|  ravi|   Finance|   NY| 79000| 53|15000|
| Dhana| Marketing|   CA| 80000| 25|18000|
| Bannu|     Sales|   CA| 81000| 30|23000|
|  Raju|   Finance|   NY| 83000| 36|19000|
|  Mona|     Sales|   NY| 86000| 56|20000|
|  Sona|     Sales|   NY| 90000| 34|10000|
|Chinnu|   Finance|   CA| 90000| 24|23000|
|   Ram| Marketing|   NY| 91000| 50|21000|
| Sunny|   Finance|   CA| 99000| 40|24000|
+------+----------+-----+------+---+-----+



In [87]:
# iv) by default sorts in ascending order
df1.sort(col("Salary"),col("Age")).show()

+------+----------+-----+------+---+-----+
|  Name|Department|State|Salary|Age|Bonus|
+------+----------+-----+------+---+-----+
|  ravi|   Finance|   NY| 79000| 53|15000|
| Dhana| Marketing|   CA| 80000| 25|18000|
| Bannu|     Sales|   CA| 81000| 30|23000|
|  Raju|   Finance|   NY| 83000| 36|19000|
|  Mona|     Sales|   NY| 86000| 56|20000|
|Chinnu|   Finance|   CA| 90000| 24|23000|
|  Sona|     Sales|   NY| 90000| 34|10000|
|   Ram| Marketing|   NY| 91000| 50|21000|
| Sunny|   Finance|   CA| 99000| 40|24000|
+------+----------+-----+------+---+-----+



In [88]:
# v)
df1.sort(df1.Salary,df1.Age).show()

+------+----------+-----+------+---+-----+
|  Name|Department|State|Salary|Age|Bonus|
+------+----------+-----+------+---+-----+
|  ravi|   Finance|   NY| 79000| 53|15000|
| Dhana| Marketing|   CA| 80000| 25|18000|
| Bannu|     Sales|   CA| 81000| 30|23000|
|  Raju|   Finance|   NY| 83000| 36|19000|
|  Mona|     Sales|   NY| 86000| 56|20000|
|Chinnu|   Finance|   CA| 90000| 24|23000|
|  Sona|     Sales|   NY| 90000| 34|10000|
|   Ram| Marketing|   NY| 91000| 50|21000|
| Sunny|   Finance|   CA| 99000| 40|24000|
+------+----------+-----+------+---+-----+



In [89]:
# 2) orderBy()  ----- by default sorts in ascending order

df1.orderBy("Name").show()

+------+----------+-----+------+---+-----+
|  Name|Department|State|Salary|Age|Bonus|
+------+----------+-----+------+---+-----+
| Bannu|     Sales|   CA| 81000| 30|23000|
|Chinnu|   Finance|   CA| 90000| 24|23000|
| Dhana| Marketing|   CA| 80000| 25|18000|
|  Mona|     Sales|   NY| 86000| 56|20000|
|  Raju|   Finance|   NY| 83000| 36|19000|
|   Ram| Marketing|   NY| 91000| 50|21000|
|  Sona|     Sales|   NY| 90000| 34|10000|
| Sunny|   Finance|   CA| 99000| 40|24000|
|  ravi|   Finance|   NY| 79000| 53|15000|
+------+----------+-----+------+---+-----+



In [90]:
# multiple columns using orderBy()

df1.orderBy("Department","Salary").show()

+------+----------+-----+------+---+-----+
|  Name|Department|State|Salary|Age|Bonus|
+------+----------+-----+------+---+-----+
|  ravi|   Finance|   NY| 79000| 53|15000|
|  Raju|   Finance|   NY| 83000| 36|19000|
|Chinnu|   Finance|   CA| 90000| 24|23000|
| Sunny|   Finance|   CA| 99000| 40|24000|
| Dhana| Marketing|   CA| 80000| 25|18000|
|   Ram| Marketing|   NY| 91000| 50|21000|
| Bannu|     Sales|   CA| 81000| 30|23000|
|  Mona|     Sales|   NY| 86000| 56|20000|
|  Sona|     Sales|   NY| 90000| 34|10000|
+------+----------+-----+------+---+-----+



In [91]:
df1.orderBy(col("Department"),col("Salary")).show()

+------+----------+-----+------+---+-----+
|  Name|Department|State|Salary|Age|Bonus|
+------+----------+-----+------+---+-----+
|  ravi|   Finance|   NY| 79000| 53|15000|
|  Raju|   Finance|   NY| 83000| 36|19000|
|Chinnu|   Finance|   CA| 90000| 24|23000|
| Sunny|   Finance|   CA| 99000| 40|24000|
| Dhana| Marketing|   CA| 80000| 25|18000|
|   Ram| Marketing|   NY| 91000| 50|21000|
| Bannu|     Sales|   CA| 81000| 30|23000|
|  Mona|     Sales|   NY| 86000| 56|20000|
|  Sona|     Sales|   NY| 90000| 34|10000|
+------+----------+-----+------+---+-----+



In [99]:
# 3.Sort by Ascending (ASC)
from pyspark.sql.functions import asc,desc


In [102]:
df1.sort(df1.Department.asc()).show()

+------+----------+-----+------+---+-----+
|  Name|Department|State|Salary|Age|Bonus|
+------+----------+-----+------+---+-----+
|Chinnu|   Finance|   CA| 90000| 24|23000|
|  ravi|   Finance|   NY| 79000| 53|15000|
| Sunny|   Finance|   CA| 99000| 40|24000|
|  Raju|   Finance|   NY| 83000| 36|19000|
| Dhana| Marketing|   CA| 80000| 25|18000|
|   Ram| Marketing|   NY| 91000| 50|21000|
|  Sona|     Sales|   NY| 90000| 34|10000|
| Bannu|     Sales|   CA| 81000| 30|23000|
|  Mona|     Sales|   NY| 86000| 56|20000|
+------+----------+-----+------+---+-----+



In [104]:
df1.sort(df1.Department.asc(),df1.Age.asc()).show()

+------+----------+-----+------+---+-----+
|  Name|Department|State|Salary|Age|Bonus|
+------+----------+-----+------+---+-----+
|Chinnu|   Finance|   CA| 90000| 24|23000|
|  Raju|   Finance|   NY| 83000| 36|19000|
| Sunny|   Finance|   CA| 99000| 40|24000|
|  ravi|   Finance|   NY| 79000| 53|15000|
| Dhana| Marketing|   CA| 80000| 25|18000|
|   Ram| Marketing|   NY| 91000| 50|21000|
| Bannu|     Sales|   CA| 81000| 30|23000|
|  Sona|     Sales|   NY| 90000| 34|10000|
|  Mona|     Sales|   NY| 86000| 56|20000|
+------+----------+-----+------+---+-----+



In [108]:
df1.sort(col("Department").asc(),col("State").asc()).show(truncate=False)

+------+----------+-----+------+---+-----+
|Name  |Department|State|Salary|Age|Bonus|
+------+----------+-----+------+---+-----+
|Chinnu|Finance   |CA   |90000 |24 |23000|
|Sunny |Finance   |CA   |99000 |40 |24000|
|Raju  |Finance   |NY   |83000 |36 |19000|
|ravi  |Finance   |NY   |79000 |53 |15000|
|Dhana |Marketing |CA   |80000 |25 |18000|
|Ram   |Marketing |NY   |91000 |50 |21000|
|Bannu |Sales     |CA   |81000 |30 |23000|
|Mona  |Sales     |NY   |86000 |56 |20000|
|Sona  |Sales     |NY   |90000 |34 |10000|
+------+----------+-----+------+---+-----+



In [109]:
df1.orderBy(col("Department").asc(),col("State").asc(),col("Salary").asc()).show()

+------+----------+-----+------+---+-----+
|  Name|Department|State|Salary|Age|Bonus|
+------+----------+-----+------+---+-----+
|Chinnu|   Finance|   CA| 90000| 24|23000|
| Sunny|   Finance|   CA| 99000| 40|24000|
|  ravi|   Finance|   NY| 79000| 53|15000|
|  Raju|   Finance|   NY| 83000| 36|19000|
| Dhana| Marketing|   CA| 80000| 25|18000|
|   Ram| Marketing|   NY| 91000| 50|21000|
| Bannu|     Sales|   CA| 81000| 30|23000|
|  Mona|     Sales|   NY| 86000| 56|20000|
|  Sona|     Sales|   NY| 90000| 34|10000|
+------+----------+-----+------+---+-----+



In [111]:
# # 4.Sort by Descending (DESC)

from pyspark.sql.functions import desc
df1.sort(df1.Department.desc()).show()

+------+----------+-----+------+---+-----+
|  Name|Department|State|Salary|Age|Bonus|
+------+----------+-----+------+---+-----+
| Bannu|     Sales|   CA| 81000| 30|23000|
|  Sona|     Sales|   NY| 90000| 34|10000|
|  Mona|     Sales|   NY| 86000| 56|20000|
| Dhana| Marketing|   CA| 80000| 25|18000|
|   Ram| Marketing|   NY| 91000| 50|21000|
|Chinnu|   Finance|   CA| 90000| 24|23000|
|  ravi|   Finance|   NY| 79000| 53|15000|
| Sunny|   Finance|   CA| 99000| 40|24000|
|  Raju|   Finance|   NY| 83000| 36|19000|
+------+----------+-----+------+---+-----+



In [112]:
df1.sort(df1.Department.desc(),df1.Age.desc()).show()

+------+----------+-----+------+---+-----+
|  Name|Department|State|Salary|Age|Bonus|
+------+----------+-----+------+---+-----+
|  Mona|     Sales|   NY| 86000| 56|20000|
|  Sona|     Sales|   NY| 90000| 34|10000|
| Bannu|     Sales|   CA| 81000| 30|23000|
|   Ram| Marketing|   NY| 91000| 50|21000|
| Dhana| Marketing|   CA| 80000| 25|18000|
|  ravi|   Finance|   NY| 79000| 53|15000|
| Sunny|   Finance|   CA| 99000| 40|24000|
|  Raju|   Finance|   NY| 83000| 36|19000|
|Chinnu|   Finance|   CA| 90000| 24|23000|
+------+----------+-----+------+---+-----+



In [113]:
df1.sort(col("Department").desc(),col("Bonus").desc()).show()

+------+----------+-----+------+---+-----+
|  Name|Department|State|Salary|Age|Bonus|
+------+----------+-----+------+---+-----+
| Bannu|     Sales|   CA| 81000| 30|23000|
|  Mona|     Sales|   NY| 86000| 56|20000|
|  Sona|     Sales|   NY| 90000| 34|10000|
|   Ram| Marketing|   NY| 91000| 50|21000|
| Dhana| Marketing|   CA| 80000| 25|18000|
| Sunny|   Finance|   CA| 99000| 40|24000|
|Chinnu|   Finance|   CA| 90000| 24|23000|
|  Raju|   Finance|   NY| 83000| 36|19000|
|  ravi|   Finance|   NY| 79000| 53|15000|
+------+----------+-----+------+---+-----+



In [114]:
# using orderBy()
df1.orderBy(col("Department").desc(),col("State").desc(),col("Age").desc()).show()

+------+----------+-----+------+---+-----+
|  Name|Department|State|Salary|Age|Bonus|
+------+----------+-----+------+---+-----+
|  Mona|     Sales|   NY| 86000| 56|20000|
|  Sona|     Sales|   NY| 90000| 34|10000|
| Bannu|     Sales|   CA| 81000| 30|23000|
|   Ram| Marketing|   NY| 91000| 50|21000|
| Dhana| Marketing|   CA| 80000| 25|18000|
|  ravi|   Finance|   NY| 79000| 53|15000|
|  Raju|   Finance|   NY| 83000| 36|19000|
| Sunny|   Finance|   CA| 99000| 40|24000|
|Chinnu|   Finance|   CA| 90000| 24|23000|
+------+----------+-----+------+---+-----+



In [119]:
######### Joining pyspark dataframes

emp = [(1,"Smith",-1,"2018","10","M",3000), 
    (2,"Rose",1,"2010","20","M",4000), 
    (3,"Williams",1,"2010","10","M",1000), 
    (4,"Jones",2,"2005","10","F",2000), 
    (5,"Brown",2,"2010","40","",-1), 
      (6,"Brown",2,"2010","50","",-1) 
    ]
empColumns = ["emp_id","name","superior_emp_id","year_joined","emp_dept_id","gender","salary"]
empDF=spark.createDataFrame(data=emp,schema=empColumns)

In [120]:
empDF

DataFrame[emp_id: bigint, name: string, superior_emp_id: bigint, year_joined: string, emp_dept_id: string, gender: string, salary: bigint]

In [121]:
print(type(empDF))

<class 'pyspark.sql.dataframe.DataFrame'>


In [122]:
empDF.show()

+------+--------+---------------+-----------+-----------+------+------+
|emp_id|    name|superior_emp_id|year_joined|emp_dept_id|gender|salary|
+------+--------+---------------+-----------+-----------+------+------+
|     1|   Smith|             -1|       2018|         10|     M|  3000|
|     2|    Rose|              1|       2010|         20|     M|  4000|
|     3|Williams|              1|       2010|         10|     M|  1000|
|     4|   Jones|              2|       2005|         10|     F|  2000|
|     5|   Brown|              2|       2010|         40|      |    -1|
|     6|   Brown|              2|       2010|         50|      |    -1|
+------+--------+---------------+-----------+-----------+------+------+



In [123]:
dept = [("Finance",10), 
    ("Marketing",20), 
    ("Sales",30), 
    ("IT",40) 
  ]
deptColumns = ["dept_name","dept_id"]
deptDF=spark.createDataFrame(data=dept,schema=deptColumns)
deptDF

DataFrame[dept_name: string, dept_id: bigint]

In [124]:
deptDF.printSchema()

root
 |-- dept_name: string (nullable = true)
 |-- dept_id: long (nullable = true)



In [125]:
deptDF.show()

+---------+-------+
|dept_name|dept_id|
+---------+-------+
|  Finance|     10|
|Marketing|     20|
|    Sales|     30|
|       IT|     40|
+---------+-------+



In [128]:
## 1) Inner Join
empDF.join(deptDF,empDF.emp_dept_id==deptDF.dept_id,"inner").show()

+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|    name|superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_name|dept_id|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|     1|   Smith|             -1|       2018|         10|     M|  3000|  Finance|     10|
|     3|Williams|              1|       2010|         10|     M|  1000|  Finance|     10|
|     4|   Jones|              2|       2005|         10|     F|  2000|  Finance|     10|
|     2|    Rose|              1|       2010|         20|     M|  4000|Marketing|     20|
|     5|   Brown|              2|       2010|         40|      |    -1|       IT|     40|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+



In [130]:
# 2) Full outer join
empDF.join(deptDF,empDF.emp_dept_id==deptDF.dept_id,"outer").show()
                # or
empDF.join(deptDF,empDF.emp_dept_id==deptDF.dept_id,"full").show()
                # or
empDF.join(deptDF,empDF.emp_dept_id==deptDF.dept_id,"fullouter").show()

+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|    name|superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_name|dept_id|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|     1|   Smith|             -1|       2018|         10|     M|  3000|  Finance|     10|
|     3|Williams|              1|       2010|         10|     M|  1000|  Finance|     10|
|     4|   Jones|              2|       2005|         10|     F|  2000|  Finance|     10|
|     2|    Rose|              1|       2010|         20|     M|  4000|Marketing|     20|
|  NULL|    NULL|           NULL|       NULL|       NULL|  NULL|  NULL|    Sales|     30|
|     5|   Brown|              2|       2010|         40|      |    -1|       IT|     40|
|     6|   Brown|              2|       2010|         50|      |    -1|     NULL|   NULL|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+

+------+-

In [133]:
# 3) Left outer join or Left join
empDF.join(deptDF,empDF.emp_dept_id==deptDF.dept_id,"left").show()

                 # or
empDF.join(deptDF,empDF.emp_dept_id==deptDF.dept_id,"leftouter").show()

+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|    name|superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_name|dept_id|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|     1|   Smith|             -1|       2018|         10|     M|  3000|  Finance|     10|
|     2|    Rose|              1|       2010|         20|     M|  4000|Marketing|     20|
|     3|Williams|              1|       2010|         10|     M|  1000|  Finance|     10|
|     4|   Jones|              2|       2005|         10|     F|  2000|  Finance|     10|
|     5|   Brown|              2|       2010|         40|      |    -1|       IT|     40|
|     6|   Brown|              2|       2010|         50|      |    -1|     NULL|   NULL|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+

+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id| 

In [142]:
# 4) Right Outer join or Right Join
empDF.join(deptDF,empDF.emp_dept_id==deptDF.dept_id,"right").show()
                
empDF.join(deptDF,empDF.emp_dept_id==deptDF.dept_id,"rightouter").show()

+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|    name|superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_name|dept_id|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|     4|   Jones|              2|       2005|         10|     F|  2000|  Finance|     10|
|     3|Williams|              1|       2010|         10|     M|  1000|  Finance|     10|
|     1|   Smith|             -1|       2018|         10|     M|  3000|  Finance|     10|
|     2|    Rose|              1|       2010|         20|     M|  4000|Marketing|     20|
|  NULL|    NULL|           NULL|       NULL|       NULL|  NULL|  NULL|    Sales|     30|
|     5|   Brown|              2|       2010|         40|      |    -1|       IT|     40|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+

+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id| 

In [143]:
# 5) left semi join
#returns columns from the only left dataset for the records match in the right dataset on join expression, 
#records not matched on join expression are ignored from both left and right datasets.

empDF.join(deptDF,empDF.emp_dept_id==deptDF.dept_id,"leftsemi").show()

+------+--------+---------------+-----------+-----------+------+------+
|emp_id|    name|superior_emp_id|year_joined|emp_dept_id|gender|salary|
+------+--------+---------------+-----------+-----------+------+------+
|     1|   Smith|             -1|       2018|         10|     M|  3000|
|     3|Williams|              1|       2010|         10|     M|  1000|
|     4|   Jones|              2|       2005|         10|     F|  2000|
|     2|    Rose|              1|       2010|         20|     M|  4000|
|     5|   Brown|              2|       2010|         40|      |    -1|
+------+--------+---------------+-----------+-----------+------+------+



In [145]:
# 6) Left Anti Join
#leftanti join does the exact opposite of the leftsemi
#leftanti join returns only columns from the left dataset for non-matched records.
empDF.join(deptDF,empDF.emp_dept_id==deptDF.dept_id,"leftanti").show()

+------+-----+---------------+-----------+-----------+------+------+
|emp_id| name|superior_emp_id|year_joined|emp_dept_id|gender|salary|
+------+-----+---------------+-----------+-----------+------+------+
|     6|Brown|              2|       2010|         50|      |    -1|
+------+-----+---------------+-----------+-----------+------+------+



In [146]:
# 7) self join

In [147]:
############## Updating
# using the previous data used in the starting 
data=[("Sona","Sales","NY",90000,34,10000),
      ("Mona","Sales","NY",86000,56,20000),
      ("Bannu","Sales","CA",81000,30,23000),
      ("Chinnu","Finance","CA",90000,24,23000),
    ("Sunny","Finance","CA",99000,40,24000),
    ("Raju","Finance","NY",83000,36,19000),
    ("ravi","Finance","NY",79000,53,15000),
    ("Dhana","Marketing","CA",80000,25,18000),
    ("Ram","Marketing","NY",91000,50,21000)]

columns=["Name","Department","State","Salary","Age","Bonus"]

df1=spark.createDataFrame(data=data,schema=columns)


In [148]:
df1.show()

+------+----------+-----+------+---+-----+
|  Name|Department|State|Salary|Age|Bonus|
+------+----------+-----+------+---+-----+
|  Sona|     Sales|   NY| 90000| 34|10000|
|  Mona|     Sales|   NY| 86000| 56|20000|
| Bannu|     Sales|   CA| 81000| 30|23000|
|Chinnu|   Finance|   CA| 90000| 24|23000|
| Sunny|   Finance|   CA| 99000| 40|24000|
|  Raju|   Finance|   NY| 83000| 36|19000|
|  ravi|   Finance|   NY| 79000| 53|15000|
| Dhana| Marketing|   CA| 80000| 25|18000|
|   Ram| Marketing|   NY| 91000| 50|21000|
+------+----------+-----+------+---+-----+



In [149]:
# 1) updating a column
df1.withColumn("Age",df1.Age/2).show()

+------+----------+-----+------+----+-----+
|  Name|Department|State|Salary| Age|Bonus|
+------+----------+-----+------+----+-----+
|  Sona|     Sales|   NY| 90000|17.0|10000|
|  Mona|     Sales|   NY| 86000|28.0|20000|
| Bannu|     Sales|   CA| 81000|15.0|23000|
|Chinnu|   Finance|   CA| 90000|12.0|23000|
| Sunny|   Finance|   CA| 99000|20.0|24000|
|  Raju|   Finance|   NY| 83000|18.0|19000|
|  ravi|   Finance|   NY| 79000|26.5|15000|
| Dhana| Marketing|   CA| 80000|12.5|18000|
|   Ram| Marketing|   NY| 91000|25.0|21000|
+------+----------+-----+------+----+-----+



In [150]:
# 2) updating column based on condition using when and otherwise
df1.withColumn("Department",when(df1.Department.startswith("S"),"Developer")
              .when(df1.Department.startswith("F"),"Tester")
              .when(df1.Department.startswith("M"),"Manager")
              .otherwise(df1.Department)).show()

+------+----------+-----+------+---+-----+
|  Name|Department|State|Salary|Age|Bonus|
+------+----------+-----+------+---+-----+
|  Sona| Developer|   NY| 90000| 34|10000|
|  Mona| Developer|   NY| 86000| 56|20000|
| Bannu| Developer|   CA| 81000| 30|23000|
|Chinnu|    Tester|   CA| 90000| 24|23000|
| Sunny|    Tester|   CA| 99000| 40|24000|
|  Raju|    Tester|   NY| 83000| 36|19000|
|  ravi|    Tester|   NY| 79000| 53|15000|
| Dhana|   Manager|   CA| 80000| 25|18000|
|   Ram|   Manager|   NY| 91000| 50|21000|
+------+----------+-----+------+---+-----+



In [159]:
# 3)Update DataFrame Column Data Type
x=df1.withColumn("Age", col("Age").cast("string"))
x.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Department: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Salary: long (nullable = true)
 |-- Age: string (nullable = true)
 |-- Bonus: long (nullable = true)



In [160]:
# 4)Update DataFrame Column with a constant
df1.withColumn("Age",lit(45)).show()

+------+----------+-----+------+---+-----+
|  Name|Department|State|Salary|Age|Bonus|
+------+----------+-----+------+---+-----+
|  Sona|     Sales|   NY| 90000| 45|10000|
|  Mona|     Sales|   NY| 86000| 45|20000|
| Bannu|     Sales|   CA| 81000| 45|23000|
|Chinnu|   Finance|   CA| 90000| 45|23000|
| Sunny|   Finance|   CA| 99000| 45|24000|
|  Raju|   Finance|   NY| 83000| 45|19000|
|  ravi|   Finance|   NY| 79000| 45|15000|
| Dhana| Marketing|   CA| 80000| 45|18000|
|   Ram| Marketing|   NY| 91000| 45|21000|
+------+----------+-----+------+---+-----+



In [8]:
##### Window functions
# 1) ROW_NUMBER()

# with partition
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
windowSpec  = Window.partitionBy("department").orderBy("salary")

df1.withColumn("row_number",row_number().over(windowSpec)).show(truncate=False)

+------+----------+-----+------+---+-----+----------+
|Name  |Department|State|Salary|Age|Bonus|row_number|
+------+----------+-----+------+---+-----+----------+
|ravi  |Finance   |NY   |79000 |53 |15000|1         |
|Raju  |Finance   |NY   |83000 |36 |19000|2         |
|Chinnu|Finance   |CA   |90000 |24 |23000|3         |
|Sunny |Finance   |CA   |99000 |40 |24000|4         |
|Dhana |Marketing |CA   |80000 |25 |18000|1         |
|Ram   |Marketing |NY   |91000 |50 |21000|2         |
|Bannu |Sales     |CA   |81000 |30 |23000|1         |
|Mona  |Sales     |NY   |86000 |56 |20000|2         |
|Sona  |Sales     |NY   |90000 |34 |10000|3         |
+------+----------+-----+------+---+-----+----------+



In [12]:
# without partition 
df1.withColumn("row_number",row_number().over(Window.orderBy("Salary"))).show()

+------+----------+-----+------+---+-----+----------+
|  Name|Department|State|Salary|Age|Bonus|row_number|
+------+----------+-----+------+---+-----+----------+
|  ravi|   Finance|   NY| 79000| 53|15000|         1|
| Dhana| Marketing|   CA| 80000| 25|18000|         2|
| Bannu|     Sales|   CA| 81000| 30|23000|         3|
|  Raju|   Finance|   NY| 83000| 36|19000|         4|
|  Mona|     Sales|   NY| 86000| 56|20000|         5|
|  Sona|     Sales|   NY| 90000| 34|10000|         6|
|Chinnu|   Finance|   CA| 90000| 24|23000|         7|
|   Ram| Marketing|   NY| 91000| 50|21000|         8|
| Sunny|   Finance|   CA| 99000| 40|24000|         9|
+------+----------+-----+------+---+-----+----------+



In [14]:
# 2) RANK()
from pyspark.sql.functions import rank
windowSpec=Window.partitionBy("Department").orderBy("Salary")
df1.withColumn("RANK",rank().over(windowSpec)).show()

+------+----------+-----+------+---+-----+----+
|  Name|Department|State|Salary|Age|Bonus|RANK|
+------+----------+-----+------+---+-----+----+
|  ravi|   Finance|   NY| 79000| 53|15000|   1|
|  Raju|   Finance|   NY| 83000| 36|19000|   2|
|Chinnu|   Finance|   CA| 90000| 24|23000|   3|
| Sunny|   Finance|   CA| 99000| 40|24000|   4|
| Dhana| Marketing|   CA| 80000| 25|18000|   1|
|   Ram| Marketing|   NY| 91000| 50|21000|   2|
| Bannu|     Sales|   CA| 81000| 30|23000|   1|
|  Mona|     Sales|   NY| 86000| 56|20000|   2|
|  Sona|     Sales|   NY| 90000| 34|10000|   3|
+------+----------+-----+------+---+-----+----+



In [16]:
# without partition 
df1.withColumn("RANK",rank().over(Window.orderBy("Salary"))).show()

+------+----------+-----+------+---+-----+----+
|  Name|Department|State|Salary|Age|Bonus|RANK|
+------+----------+-----+------+---+-----+----+
|  ravi|   Finance|   NY| 79000| 53|15000|   1|
| Dhana| Marketing|   CA| 80000| 25|18000|   2|
| Bannu|     Sales|   CA| 81000| 30|23000|   3|
|  Raju|   Finance|   NY| 83000| 36|19000|   4|
|  Mona|     Sales|   NY| 86000| 56|20000|   5|
|  Sona|     Sales|   NY| 90000| 34|10000|   6|
|Chinnu|   Finance|   CA| 90000| 24|23000|   6|
|   Ram| Marketing|   NY| 91000| 50|21000|   8|
| Sunny|   Finance|   CA| 99000| 40|24000|   9|
+------+----------+-----+------+---+-----+----+



In [15]:
# 3) DENSE_RANK()
from pyspark.sql.functions import dense_rank
windowSpec=Window.partitionBy("Department").orderBy("Salary")
df1.withColumn("DENSE RANK",dense_rank().over(windowSpec)).show()

+------+----------+-----+------+---+-----+----------+
|  Name|Department|State|Salary|Age|Bonus|DENSE RANK|
+------+----------+-----+------+---+-----+----------+
|  ravi|   Finance|   NY| 79000| 53|15000|         1|
|  Raju|   Finance|   NY| 83000| 36|19000|         2|
|Chinnu|   Finance|   CA| 90000| 24|23000|         3|
| Sunny|   Finance|   CA| 99000| 40|24000|         4|
| Dhana| Marketing|   CA| 80000| 25|18000|         1|
|   Ram| Marketing|   NY| 91000| 50|21000|         2|
| Bannu|     Sales|   CA| 81000| 30|23000|         1|
|  Mona|     Sales|   NY| 86000| 56|20000|         2|
|  Sona|     Sales|   NY| 90000| 34|10000|         3|
+------+----------+-----+------+---+-----+----------+



In [17]:
# without partition 
df1.withColumn("DENSE RANK",dense_rank().over(Window.orderBy("Salary"))).show()

+------+----------+-----+------+---+-----+----------+
|  Name|Department|State|Salary|Age|Bonus|DENSE RANK|
+------+----------+-----+------+---+-----+----------+
|  ravi|   Finance|   NY| 79000| 53|15000|         1|
| Dhana| Marketing|   CA| 80000| 25|18000|         2|
| Bannu|     Sales|   CA| 81000| 30|23000|         3|
|  Raju|   Finance|   NY| 83000| 36|19000|         4|
|  Mona|     Sales|   NY| 86000| 56|20000|         5|
|  Sona|     Sales|   NY| 90000| 34|10000|         6|
|Chinnu|   Finance|   CA| 90000| 24|23000|         6|
|   Ram| Marketing|   NY| 91000| 50|21000|         7|
| Sunny|   Finance|   CA| 99000| 40|24000|         8|
+------+----------+-----+------+---+-----+----------+



In [19]:
# 4) PERCENT_RANK()
from pyspark.sql.functions import percent_rank
windowSpec=Window.partitionBy("Department").orderBy("Salary")
df1.withColumn("PERCENT RANK",percent_rank().over(windowSpec)).show()

+------+----------+-----+------+---+-----+------------------+
|  Name|Department|State|Salary|Age|Bonus|      PERCENT RANK|
+------+----------+-----+------+---+-----+------------------+
|  ravi|   Finance|   NY| 79000| 53|15000|               0.0|
|  Raju|   Finance|   NY| 83000| 36|19000|0.3333333333333333|
|Chinnu|   Finance|   CA| 90000| 24|23000|0.6666666666666666|
| Sunny|   Finance|   CA| 99000| 40|24000|               1.0|
| Dhana| Marketing|   CA| 80000| 25|18000|               0.0|
|   Ram| Marketing|   NY| 91000| 50|21000|               1.0|
| Bannu|     Sales|   CA| 81000| 30|23000|               0.0|
|  Mona|     Sales|   NY| 86000| 56|20000|               0.5|
|  Sona|     Sales|   NY| 90000| 34|10000|               1.0|
+------+----------+-----+------+---+-----+------------------+



In [20]:
# without partition 
df1.withColumn("PERCENT RANK",percent_rank().over(Window.orderBy("Salary"))).show()

+------+----------+-----+------+---+-----+------------+
|  Name|Department|State|Salary|Age|Bonus|PERCENT RANK|
+------+----------+-----+------+---+-----+------------+
|  ravi|   Finance|   NY| 79000| 53|15000|         0.0|
| Dhana| Marketing|   CA| 80000| 25|18000|       0.125|
| Bannu|     Sales|   CA| 81000| 30|23000|        0.25|
|  Raju|   Finance|   NY| 83000| 36|19000|       0.375|
|  Mona|     Sales|   NY| 86000| 56|20000|         0.5|
|  Sona|     Sales|   NY| 90000| 34|10000|       0.625|
|Chinnu|   Finance|   CA| 90000| 24|23000|       0.625|
|   Ram| Marketing|   NY| 91000| 50|21000|       0.875|
| Sunny|   Finance|   CA| 99000| 40|24000|         1.0|
+------+----------+-----+------+---+-----+------------+



In [21]:
# 5) ntile(n)
from pyspark.sql.functions import ntile
windowSpec=Window.partitionBy("Department").orderBy("Salary")
df1.withColumn("ntile",ntile(3).over(windowSpec)).show()

+------+----------+-----+------+---+-----+-----+
|  Name|Department|State|Salary|Age|Bonus|ntile|
+------+----------+-----+------+---+-----+-----+
|  ravi|   Finance|   NY| 79000| 53|15000|    1|
|  Raju|   Finance|   NY| 83000| 36|19000|    1|
|Chinnu|   Finance|   CA| 90000| 24|23000|    2|
| Sunny|   Finance|   CA| 99000| 40|24000|    3|
| Dhana| Marketing|   CA| 80000| 25|18000|    1|
|   Ram| Marketing|   NY| 91000| 50|21000|    2|
| Bannu|     Sales|   CA| 81000| 30|23000|    1|
|  Mona|     Sales|   NY| 86000| 56|20000|    2|
|  Sona|     Sales|   NY| 90000| 34|10000|    3|
+------+----------+-----+------+---+-----+-----+



In [22]:
# without partition 
df1.withColumn("NTILE",ntile(3).over(Window.orderBy("Salary"))).show()

+------+----------+-----+------+---+-----+-----+
|  Name|Department|State|Salary|Age|Bonus|NTILE|
+------+----------+-----+------+---+-----+-----+
|  ravi|   Finance|   NY| 79000| 53|15000|    1|
| Dhana| Marketing|   CA| 80000| 25|18000|    1|
| Bannu|     Sales|   CA| 81000| 30|23000|    1|
|  Raju|   Finance|   NY| 83000| 36|19000|    2|
|  Mona|     Sales|   NY| 86000| 56|20000|    2|
|  Sona|     Sales|   NY| 90000| 34|10000|    2|
|Chinnu|   Finance|   CA| 90000| 24|23000|    3|
|   Ram| Marketing|   NY| 91000| 50|21000|    3|
| Sunny|   Finance|   CA| 99000| 40|24000|    3|
+------+----------+-----+------+---+-----+-----+



In [23]:
# Window Aggregate functions 

windowSpecAgg  = Window.partitionBy("department")
from pyspark.sql.functions import col,avg,sum,min,max,row_number 

df1.withColumn("row",row_number().over(windowSpec)) \
  .withColumn("avg", avg(col("salary")).over(windowSpecAgg)) \
  .withColumn("sum", sum(col("salary")).over(windowSpecAgg)) \
  .withColumn("min", min(col("salary")).over(windowSpecAgg)) \
  .withColumn("max", max(col("salary")).over(windowSpecAgg)) \
  .where(col("row")==1).select("department","avg","sum","min","max") \
  .show()

+----------+-----------------+------+-----+-----+
|department|              avg|   sum|  min|  max|
+----------+-----------------+------+-----+-----+
|   Finance|          87750.0|351000|79000|99000|
| Marketing|          85500.0|171000|80000|91000|
|     Sales|85666.66666666667|257000|81000|90000|
+----------+-----------------+------+-----+-----+



In [ ]:
# Window Analytic functions